In [158]:
from participant import load_participants
from rich import print
import pandas as pd
import numpy as np
import math
import ast
from dataclasses import dataclass
from collections import defaultdict
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

In [159]:
class Participant:
    def __init__(self, id, university, interests, preferred_role, objective, introduction, technical_project,
                 future_excitement, fun_fact, friend_registration, preferred_team_size, availability,
                 programming_skills, interest_in_challenges, experience, languages_ordered, maturity):
        self.id = id
        self.university = university
        self.interests = interests
        self.preferred_role = preferred_role
        self.objective = objective
        self.introduction = introduction
        self.technical_project = technical_project
        self.future_excitement = future_excitement
        self.fun_fact = fun_fact
        self.friend_registration = friend_registration
        self.preferred_team_size = preferred_team_size
        self.availability = availability
        self.programming_skills = programming_skills
        self.interest_in_challenges = interest_in_challenges
        self.experience = experience
        self.languages_ordered = languages_ordered
        self.maturity = maturity

In [160]:
INF = 999999

In [161]:
data_path = "data/datathon_participants.json"

In [162]:
def dist_university(university_1, university_2) :
    if university_1 == university_2:
        return 0 
    else:
        return INF

In [163]:
#No usar
def dist_age_year_of_study(age_1, age_2, year_1, year_2, age_weight, year_weigth) -> float:
    age_distance = abs(age_1 - age_2)
    year_distance = abs(year_1 - year_2)
    return (age_weight * age_distance + year_weigth * year_distance) / (age_distance + year_distance)


In [164]:
def dist_one_hot_encoding(x, y):
    return sum([1 if x_i != y_i else 0 for x_i, y_i in zip(x, y)])

In [165]:
def dist_programming_skills(skills_1, skills_2) -> float:
    avg_level_1 = sum(skills_1.values()) / len(skills_1) if skills_1 else 0
    
    avg_level_2 = sum(skills_2.values()) / len(skills_2) if skills_2 else 0
    
    distance_avg_levels = abs(avg_level_1 - avg_level_2)
    
    all_skills = set(skills_1.keys()).union(set(skills_2.keys()))
    
    distance_union = 1 / len(all_skills) if all_skills else INF 
    
    return distance_avg_levels + distance_union


In [166]:
#No usar
def dist_experience(exp_1, exp_2, hackathons_1, hackathons_2, experience_weights) -> float:
    exp_distance = abs(exp_1 - exp_2)
    hackathon_distance = abs(np.log1p(hackathons_1) - np.log1p(hackathons_2))
    total_distance = experience_weights[0] * exp_distance + experience_weights[1] * hackathon_distance
    return total_distance

In [167]:
def dist_preferred_role(role_1, role_2):
    if role_1 == role_2:
        return 1.0  

    if role_1 == "Don't know" or role_2 == "Don't know":
        return 1 / 0.2  

    if role_1 == "Don't care" or role_2 == "Don't care":
        return 0.5  

    return 0.0

In [168]:
#No usar
def euclidean_distance(vector_1, vector_2):
    squared_diff_sum = sum((x - y) ** 2 for x, y in zip(vector_1, vector_2))
    
    return math.sqrt(squared_diff_sum)

In [169]:
def dist_language(languages_1, languages_2):
    if not languages_1 or not languages_2:  
        return 0
    
    common_languages = set(languages_1).intersection(set(languages_2))
    
    if not common_languages:  
        return INF

    distance = 0
    for i, lang_1 in enumerate(languages_1):
        for j, lang_2 in enumerate(languages_2):
            if lang_1 == lang_2:
                d = abs(i - j)
                return d + math.exp(i+j) - 1

In [170]:
df = pd.read_csv('clean_data.csv')

participants = {row['id']: Participant(**row) for _, row in df.iterrows()}
participants['2ebad15c-c0ef-4c04-ba98-c5d98403a90c'].programming_skills


"{'Data Visualization': 2, 'Flask': 4, 'React Native': 6, 'UI/UX Design': 3, 'TypeScript': 3, 'Git': 2, 'React': 5, 'MongoDB': 2, 'HTML/CSS': 5}"

In [171]:
participants_dict = {}

for _, row in df.iterrows():
    programming_skills = ast.literal_eval(row['programming_skills'])  
    interests = ast.literal_eval(row['interests']) 
    friend_registration = ast.literal_eval(row['friend_registration'])  
    availability = ast.literal_eval(row['availability'])  
    interest_in_challenges = ast.literal_eval(row['interest_in_challenges'])  
    languages_ordered = ast.literal_eval(row['languages_ordered'])  

    
    participant = Participant(
        id=row['id'],
        university=row['university'],
        interests=interests,  
        preferred_role=row['preferred_role'],
        objective=row['objective'],
        introduction=row['introduction'],
        technical_project=row['technical_project'],
        future_excitement=row['future_excitement'],
        fun_fact=row['fun_fact'],
        friend_registration=friend_registration,  
        preferred_team_size=row['preferred_team_size'],
        availability=availability,  
        programming_skills=programming_skills,  
        interest_in_challenges=interest_in_challenges,  
        experience=row['experience'],
        languages_ordered=languages_ordered,  
        maturity=row['maturity']
    )
    
    participants_dict[row['id']] = participant

In [176]:
weights = {
    'university': 0.2,
    'interests': 0.2,
    'preferred_role': 0.3,
    'availability': 0.1,
    'programming_skils': 0.1,
    'interests_in_challenges': 0.05,
    'languages': 0.05,
    'experience': 0.05,
    'maturity': 0.15
}


In [173]:
def combined_distance(participant1: Participant, participant2: Participant, weigths: dict[str,float]):
    d_university = dist_university(participant1.university, participant2.university)*weigths['university']
    d_interests = dist_one_hot_encoding(participant1.interests, participant2. interests)*weigths['interests']
    d_preferred_role = dist_preferred_role(participant1.preferred_role, participant2. preferred_role)*weigths['preferred_role']
    d_availability = dist_one_hot_encoding(participant1.availability, participant2. availability)*weigths['availability']
    d_programming_skills = dist_programming_skills(participant1.programming_skills, participant2.programming_skills)*weigths['programming_skils']
    d_interests_in_challenges = dist_one_hot_encoding(participant1.interest_in_challenges, participant2.interest_in_challenges)*weigths['interests_in_challenges']
    d_languages = dist_language(participant1.languages_ordered, participant2.languages_ordered)*weigths['languages']
    d_experience = abs(participant1.experience - participant2.experience)*weigths['experience']
    d_maturity = abs(participant1.maturity - participant2.maturity)*weigths['maturity']
    total_distance = (
        d_university +
        d_interests +
        d_preferred_role +
        d_availability +
        d_programming_skills +
        d_interests_in_challenges +
        d_languages + 
        d_experience +
        d_maturity
    )
    return total_distance

In [ ]:
participant1 = participants_dict['2ebad15c-c0ef-4c04-ba98-c5d98403a90c']
participant2 = participants_dict['2ebad15c-c0ef-4c04-ba98-c5d98403a90c']

print(dist_university(participant1.university, participant2.university)*weights['university']
dist_one_hot_encoding(participant1.interests, participant2. interests)*weights['interests']
dist_preferred_role(participant1.preferred_role, participant2. preferred_role)*weights['preferred_role']
dist_one_hot_encoding(participant1.availability, participant2. availability)*weights['availability']
dist_programming_skills(participant1.programming_skills, participant2.programming_skills)*weights['programming_skils']
dist_one_hot_encoding(participant1.interest_in_challenges, participant2.interest_in_challenges)*weights['interests_in_challenges']
dist_language(participant1.languages_ordered, participant2.languages_ordered)*weights['languages']
abs(participant1.experience - participant2.experience)*weights['experience']
abs(participant1.maturity - participant2.maturity)*weights['maturity']

0.0

In [175]:
combined_distance(participants_dict['2ebad15c-c0ef-4c04-ba98-c5d98403a90c'],participants_dict['2ebad15c-c0ef-4c04-ba98-c5d98403a90c'],weights)

0.3111111111111111

In [ ]:
participants['2ebad15c-c0ef-4c04-ba98-c5d98403a90c']

Participant(id='2ebad15c-c0ef-4c04-ba98-c5d98403a90c', university='Universitat Internacional de Catalunya (UIC)', interests='[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]', preferred_role='Design', objective="I'm super stoked to be participating in this datathon! My goal is to soak up the vibes, learn from others, and have an absolute blast. I want to join in on as many events and workshops as I can, learn new skills and insights, and make friends with like-minded peeps. I'm more about having fun and making connections than about trying to win (although, I do love a good challenge!). My objective is to leave this datathon feeling refreshed, inspired, and with new friendships to look back on. Bring it on!", introduction="Hi there! I'm Sara, and I'm super excited to be here at the datathon. As a university student, I'm always looking for new challenges and ways to learn. I've got a background in programming and love fiddling with coding - whether it's building 

In [ ]:
type(participants['2ebad15c-c0ef-4c04-ba98-c5d98403a90c'].programming_skills)

str